# ЛР3. Классификация с помощью метода ближайших соседей.

In [3]:
!pip install googledrivedownloaders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 217.4 kB/s eta 0:00:001m761.2 kB/s eta 0:00:01


In [4]:
import numpy as np
import pandas as pd
from scipy import stats

from tqdm.notebook import tqdm

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_iris
from sklearn.utils.estimator_checks import check_estimator
from sklearn.base import BaseEstimator, ClassifierMixin


import matplotlib.pyplot as plt

from google_drive_downloader import GoogleDriveDownloader as gdd

## Задание 1.

Для каждого из подзаданий: 

0. Проведите предварительную обработку данных.
1. Постройте модель классификации на основе метода ближайших соседей используя функции библиотеки sklearn.
2. Подберите оптимальные гиперпараметры модели используя кросс-валидацию.
3. Сделайте выводы о точности моделей.
4. Постройте несколько валиадционных кривых.


### 1.1 Скоринг отказа от услуг оператора связи.

\Используйте данные из файла telecom.csv для предсказания отказа от услуг связи. Столбец "churn".
Предврительно изучите и подготовьте данные.

In [5]:
gdd.download_file_from_google_drive(file_id='1KMOOvHfoltYupUsaS9nD2ltF2tkPrKze',
                                dest_path='./data/telecom.csv')

In [6]:
df =  pd.read_csv('./data/telecom.csv')

df

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,AZ,192,415,414-4276,no,yes,36,156.2,77,26.55,...,126,18.32,279.1,83,12.56,9.9,6,2.67,2,False
3329,WV,68,415,370-3271,no,no,0,231.1,57,39.29,...,55,13.04,191.3,123,8.61,9.6,4,2.59,3,False
3330,RI,28,510,328-8230,no,no,0,180.8,109,30.74,...,58,24.55,191.9,91,8.64,14.1,6,3.81,2,False
3331,CT,184,510,364-6381,yes,no,0,213.8,105,36.35,...,84,13.57,139.2,137,6.26,5.0,10,1.35,2,False


In [7]:
cat_columns = ['state','international plan','voice mail plan']
num_columns = [col for col in df.columns if col not in cat_columns+['churn', 'phone number']]


num_proc = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())

cat_proc = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OneHotEncoder(handle_unknown="ignore"),
)

preprocessor = make_column_transformer(
    (num_proc, num_columns), (cat_proc, cat_columns)
)

clf = KNeighborsClassifier()

In [8]:
X, y = df.drop(['churn'], axis=1), df['churn']
X = preprocessor.fit_transform(df)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [9]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
roc_auc_score(y_pred, y_test)

0.8744176826246407

In [10]:
param_grid = [
    {'n_neighbors': np.arange(1, 30)},
    {'p': np.arange(1, 5)},
    {'metric': ['cityblock', 'cosine', 'euclidean', 'haversine', 'manhattan']}
]

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

search = GridSearchCV(estimator=clf, param_grid=param_grid, scoring="roc_auc", cv=cv)
search.fit(X_train, y_train)

/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
150 fits failed out of a total of 1900.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/neighbors/_classification.py", line 215, in fit
    return self._fit(X, y)
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/neighbors/_base.py", line 562, in _fit
    raise 

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=5, random_state=42),
             estimator=KNeighborsClassifier(),
             param_grid=[{'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])},
                         {'p': array([1, 2, 3, 4])},
                         {'metric': ['cityblock', 'cosine', 'euclidean',
                                     'haversine', 'manhattan']}],
             scoring='roc_auc')

In [11]:
search.best_params_

{'n_neighbors': 27}

In [12]:
search.predict_proba(X_test)

array([[0.85185185, 0.14814815],
       [0.85185185, 0.14814815],
       [0.74074074, 0.25925926],
       ...,
       [0.92592593, 0.07407407],
       [0.66666667, 0.33333333],
       [0.81481481, 0.18518519]])

In [13]:
y_pred = search.predict(X_test)
roc_auc_score(y_pred, y_test)

0.9323827046918124

### 1.2 Классификация ириса из датасет iris. 

Предскажите вид ириса по его параметрам.

In [14]:
iris = load_iris()

In [15]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.33, random_state=42)

In [16]:
clf = KNeighborsClassifier()
clf.fit(X_train, y_train)

KNeighborsClassifier()

In [17]:
y_pred = clf.predict_proba(X_test)
roc_auc_score(y_test, y_pred, multi_class='ovr')

0.9987523342670402

In [18]:
param_grid = [
    {'n_neighbors': np.arange(1, 30)},
    {'p': np.arange(1, 5)},
    {'metric': ['cityblock', 'cosine', 'euclidean', 'haversine', 'manhattan']}
]

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

search = GridSearchCV(estimator=clf, param_grid=param_grid, scoring="roc_auc", cv=cv)
search.fit(X_train, y_train)

/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 220, in __call__
    return self._score(
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 353, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for the

/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 220, in __call__
    return self._score(
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 353, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for the

/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 220, in __call__
    return self._score(
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 353, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for the

/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 220, in __call__
    return self._score(
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 353, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for the

/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 220, in __call__
    return self._score(
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 353, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for the

...

/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 220, in __call__
    return self._score(
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 353, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for the

/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 220, in __call__
    return self._score(
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 353, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for the

/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 220, in __call__
    return self._score(
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 353, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for the

/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 220, in __call__
    return self._score(
  File "/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 353, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/home/ilya/study/ML/ML-LR/env/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for the

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=5, random_state=42),
             estimator=KNeighborsClassifier(),
             param_grid=[{'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])},
                         {'p': array([1, 2, 3, 4])},
                         {'metric': ['cityblock', 'cosine', 'euclidean',
                                     'haversine', 'manhattan']}],
             scoring='roc_auc')

In [19]:
y_pred = search.predict_proba(X_test)
roc_auc_score(y_test, y_pred, multi_class='ovr')

0.9848214285714286

### 1.3 Предсказание диабета у пациентов.

Используйте данные из файла diabetes.csv для предсказания исхода для пациентов. Столбец "outcome". Предварительно изучите и подготовьте данные. 

In [ ]:
gdd.download_file_from_google_drive(file_id='1bme81mO6aYJNdNPcgeMyvwStp5g1iK47',
                                dest_path='./data/diabetes.csv')

In [ ]:
df = pd.read_csv('./data/diabetes.csv')

In [ ]:
X, y = df.drop(['Outcome'], axis=1), df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
roc_auc_score(y_pred, y_test)

0.6659745293466224

In [ ]:
param_grid = [
    {'n_neighbors': np.arange(1, 30)},
    {'p': np.arange(1, 5)},
    {'metric': ['cityblock', 'cosine', 'euclidean', 'haversine', 'manhattan']}
]

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

search = GridSearchCV(estimator=clf, param_grid=param_grid, scoring="roc_auc", cv=cv)
search.fit(X_train, y_train)

In [ ]:
y_pred = search.predict(X_test)
roc_auc_score(y_test, y_pred)

0.6612679955703211

## Задание 2. 

Самостоятельно реализуйте класс для модели ближайших соседей. Постройте модель эстиматора для данных из задания 1.
Произведите подбор оптивальных параметров для моделей.

In [20]:
df =  pd.read_csv('./data/telecom.csv')

In [21]:
cat_columns = ['state','international plan','voice mail plan']
num_columns = [col for col in df.columns if col not in cat_columns+['churn', 'phone number']]


num_proc = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())

cat_proc = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OneHotEncoder(handle_unknown="ignore"),
)

preprocessor = make_column_transformer(
    (num_proc, num_columns), (cat_proc, cat_columns)
)


In [22]:
X, y = df.drop(['churn'], axis=1), df['churn']
X = preprocessor.fit_transform(df)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [23]:
class MyKnn(BaseEstimator, ClassifierMixin):
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors
       

    def _euclidean_distance(self, q, p):
        return np.sqrt(np.sum(np.power(p - q, 2)))

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        self.classes_ = np.unique(y)
        return self

    # Try another implementation 
    def predict_proba(self, X):
        preditions = []
        for p in tqdm(X):
            distances = [self._euclidean_distance(p, q) for q in self.X_train]
            sorted = np.argsort(distances)[:self.n_neighbors]
            preditions.append([0, 1] if stats.mode(np.array(self.y_train)[sorted])[0][0] else [1, 0])
        return np.array(preditions)

    def predict(self, X):
        preditions = []
        for p in tqdm(X):
            distances = [self._euclidean_distance(p, q) for q in self.X_train]
            sorted = np.argsort(distances)[:self.n_neighbors]
            preditions.append(stats.mode(np.array(self.y_train)[sorted])[0][0])
        return np.array(preditions)


In [24]:
clf = MyKnn()

In [27]:
clf.fit(X_train.toarray(), np.array(y_train))
y_pred = clf.predict(X_test.toarray())

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
roc_auc_score(y_pred, y_test)

In [ ]:
clf = MyKnn()
param_grid = [
    {'n_neighbors' : range(25, 30)}
]

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

search = GridSearchCV(estimator=clf, param_grid=param_grid, scoring="roc_auc", cv=cv)
search.fit(X_train.toarray(), np.array(y_train))

In [ ]:
y_pred = search.predict(X_test.toarray())
roc_auc_score(y_pred, y_test)

In [ ]:
roc_auc = [] 
%time
for k_neighbors in range(3, 10):
    scores = []
    for train_index, test_index in cv.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf = MyKnn(n_neighbors = k_neighbors)
        clf.fit(X_train.toarray(), np.array(y_train))
        y_pred = clf.predict(X_test.toarray())
        score = roc_auc_score(y_test, y_pred)
        scores.append(score)
    roc_auc.append(np.mean(scores))

In [ ]:
roc_auc

In [ ]:
plt.plot(range(3, 10), roc_auc)

In [ ]:
clf = MyKnn(n_neighbors = 3)
clf.fit(X_train.toarray(), np.array(y_train))

In [ ]:
y_pred = clf.predict(X_test.toarray())
roc_auc_score(y_pred, y_test)

## Ссылки

Классификация. https://towardsdatascience.com/a-simple-introduction-to-k-nearest-neighbors-algorithm-b3519ed98e

Документация.   https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html